In [1]:
# Packages
import numpy as np
import pandas as pd

from google.colab import files
uploaded = files.upload()

Saving EXPORT.XLSX to EXPORT.XLSX


In [2]:
# Data Warehouse Task
data = pd.read_excel('EXPORT.XLSX')
data.head()

,Warehouse Order,Warehouse Task,Queue,Activity,WO Activity Area,Source Storage Bin,Dest. Stor. Bin,Warehouse Task Status,Source Handling Unit,Act. Qty Dest. BUoM,Src Trgt Qty BUoM,HU Warehouse Task,Product,Batch,Shelf Life Exp. Date,Whse Proc. Type,Whse Proc. Cat.,Warehouse Process Cat. Desc.,Base Unit of Measure,Source Storage Type,Source Resource,Original Dest. Bin,Dest. Storage Type,Dest. Handling Unit,Loading Weight,Weight Unit,Loading Volume,Volume Unit,Created By,Created On,Created At,Start Date,Start Time,Confirmed by,Confirmation Date,Confirmation Time,Dest.Diff.Qty in BUM,Src Trgt Qty AUoM,Act.Qty Dest. Alt.Un,Diff. Qty in Alt.Un,Alt. Unit of Measure
0,13870874.0,25443823.0,REPL,INTL,NaN,NaN,RF-54-10,C,3.930267e+17,4,4,NaN,971022.0,2112122593,2022-06-10 00:00:00,9999.0,3.0,Internal Warehouse Movement,CS,NaN,FLS16,RF-54-10,PICK,NaN,80.0,KG,0.146,M3,IDUMAMMR,2022-01-08,00:14:57,2022-01-08,00:14:57,IDUMAMMR,2022-01-08,00:14:57,0.0,4.0,4.0,0.0,CS
1,13871700.0,25445127.0,REPL,INTL,NaN,NaN,RG-38-10,C,3.930267e+17,15,15,NaN,970057.0,2112172005,2022-12-17 00:00:00,9999.0,3.0,Internal Warehouse Movement,CS,NaN,FLS16,RG-38-10,PICK,NaN,105.0,KG,0.139,M3,IDSAKTIZZ,2022-01-08,00:55:07,2022-01-08,00:55:07,IDSAKTIZZ,2022-01-08,00:55:07,0.0,15.0,15.0,0.0,CS
2,13873465.0,25448726.0,REPL,INTL,NaN,NaN,RH-19-10,C,3.930267e+17,7,7,NaN,971025.0,2112142002,2022-06-12 00:00:00,9999.0,3.0,Internal Warehouse Movement,CS,NaN,FLS16,RH-19-10,PICK,NaN,37.1,KG,0.065,M3,IDUMAMMR,2022-01-08,04:12:15,2022-01-08,04:12:15,IDUMAMMR,2022-01-08,04:12:15,0.0,7.0,7.0,0.0,CS
3,13879432.0,25458465.0,REPL,INTL,NaN,NaN,RB-16-10,C,3.930267e+17,39,39,NaN,970293.0,2112232007,2023-04-22 00:00:00,9999.0,3.0,Internal Warehouse Movement,CS,NaN,FLS17,RB-16-10,PICD,NaN,585.0,KG,0.992,M3,IDADHITAMAN,2022-01-08,15:11:48,2022-01-08,15:11:48,IDADHITAMAN,2022-01-08,15:11:48,0.0,39.0,39.0,0.0,CS
4,13880016.0,25459227.0,REPL,INTL,PND,XB-01-00,RC-22-10,C,3.930267e+17,30,30,X,970926.0,2201082592,2022-10-05 00:00:00,9999.0,3.0,Internal Warehouse Movement,CS,PND,NaN,RC-22-10,PICD,3.930267e+17,625.0,KG,1.115,M3,IDADHITAMAN,2022-01-08,16:00:41,2022-01-08,16:00:41,IDADHITAMAN,2022-01-08,16:00:41,0.0,30.0,30.0,0.0,CS


In [4]:
# Drop unnecessary columns
data.drop(data.columns.difference(['Queue','Source Storage Bin', 'Dest. Stor. Bin', 'Act. Qty Dest. BUoM', 'Src Trgt Qty BUoM', 'Product', 'Batch', 'Shelf Life Exp. Date', 'Confirmation Date', 'Dest. Storage Type']), 1, inplace=True)
data.head()

,Queue,Source Storage Bin,Dest. Stor. Bin,Act. Qty Dest. BUoM,Src Trgt Qty BUoM,Product,Batch,Shelf Life Exp. Date,Dest. Storage Type,Confirmation Date
0,REPL,NaN,RF-54-10,4,4,971022.0,2112122593,2022-06-10 00:00:00,PICK,2022-01-08
1,REPL,NaN,RG-38-10,15,15,970057.0,2112172005,2022-12-17 00:00:00,PICK,2022-01-08
2,REPL,NaN,RH-19-10,7,7,971025.0,2112142002,2022-06-12 00:00:00,PICK,2022-01-08
3,REPL,NaN,RB-16-10,39,39,970293.0,2112232007,2023-04-22 00:00:00,PICD,2022-01-08
4,REPL,XB-01-00,RC-22-10,30,30,970926.0,2201082592,2022-10-05 00:00:00,PICD,2022-01-08


In [5]:
# Create dummy columns to join warehouse task
data['Dest_Storage_shift'] = data['Dest. Stor. Bin'].shift(-1, axis=0)
data['Product_shift'] = data['Product'].shift(-1, axis=0)
data['SLED_shift'] = data['Shelf Life Exp. Date'].shift(-1, axis=0)
data['Dest_Type_shift'] = data['Dest. Storage Type'].shift(-1, axis=0)

In [6]:
# Translate condition to columns
data.loc[data['Source Storage Bin'] == data['Dest_Storage_shift'], 'cond1'] = 1 # same bin transaction
data.loc[data['Source Storage Bin'] != data['Dest_Storage_shift'], 'cond1'] = 0 # different bin transaction
data.loc[data['Product'] == data['Product_shift'], 'cond2'] = 1 # same product
data.loc[data['Product'] != data['Product_shift'], 'cond2'] = 0 # different product
data.loc[data['Shelf Life Exp. Date'] != data['SLED_shift'], 'cond3'] = 1 # changed SLED
data.loc[data['Shelf Life Exp. Date'] == data['SLED_shift'], 'cond3'] = 0 # same SLED

In [7]:
# Create column with final score where cond1 and cond2 indicate same WT and cond3 indicate changed SLED
data['Final_Category'] = data['cond1'] + data['cond2'] + data['cond3']
data.loc[data['Final_Category'] == 3]

# Fill blank columns with empty
data = data.fillna('Empty')
data

,Queue,Source Storage Bin,Dest. Stor. Bin,Act. Qty Dest. BUoM,Src Trgt Qty BUoM,Product,Batch,Shelf Life Exp. Date,Dest. Storage Type,Confirmation Date,Dest_Storage_shift,Product_shift,SLED_shift,Dest_Type_shift,cond1,cond2,cond3,Final_Category
0,REPL,Empty,RF-54-10,4,4,971022,2112122593,2022-06-10 00:00:00,PICK,2022-01-08 00:00:00,RG-38-10,970057,2022-12-17 00:00:00,PICK,0.0,0.0,1.0,1.0
1,REPL,Empty,RG-38-10,15,15,970057,2112172005,2022-12-17 00:00:00,PICK,2022-01-08 00:00:00,RH-19-10,971025,2022-06-12 00:00:00,PICK,0.0,0.0,1.0,1.0
2,REPL,Empty,RH-19-10,7,7,971025,2112142002,2022-06-12 00:00:00,PICK,2022-01-08 00:00:00,RB-16-10,970293,2023-04-22 00:00:00,PICD,0.0,0.0,1.0,1.0
3,REPL,Empty,RB-16-10,39,39,970293,2112232007,2023-04-22 00:00:00,PICD,2022-01-08 00:00:00,RC-22-10,970926,2022-10-05 00:00:00,PICD,0.0,0.0,1.0,1.0
4,REPL,XB-01-00,RC-22-10,30,30,970926,2201082592,2022-10-05 00:00:00,PICD,2022-01-08 00:00:00,RC-23-10,970926,2022-10-05 00:00:00,PICD,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72927,Empty,Empty,INT-TRANSIT-DC,100,0,971040,2201112002,2022-07-10 00:00:00,ITRN,2022-01-11 00:00:00,Empty,970306,2023-04-15 00:00:00,Empty,0.0,0.0,1.0,1.0
72928,Empty,Empty,Empty,33,0,970306,9000033594,2023-04-15 00:00:00,Empty,2022-01-11 00:00:00,Empty,970306,2023-04-15 00:00:00,Empty,0.0,1.0,0.0,1.0
72929,Empty,Empty,Empty,33,0,970306,9000033594,2023-04-15 00:00:00,Empty,2022-01-11 00:00:00,Empty,Empty,Empty,Empty,0.0,0.0,1.0,1.0
72930,Empty,Empty,Empty,4551223,2549463,Empty,Empty,Empty,Empty,Empty,Empty,Empty,Empty,Empty,0.0,0.0,1.0,1.0


In [8]:
# Made new empty dataframe
must_check = pd.DataFrame()
must_check

""


In [9]:
# Append internal movement data to dataframe for changed SLED because of internal
data_check = data[data.Queue == 'INTERNAL']
data_check = data_check.loc[data_check['Final_Category'] == 3]
data_check = data_check.drop_duplicates(subset=['Source Storage Bin'], keep='last')
must_check = must_check.append(data_check)

In [10]:
# Append replenish movement data to dataframe for changed SLED because of replenish
data_check = data[data.Queue == 'REPL']
data_check = data_check.loc[data_check['Final_Category'] == 3]
data_check = data_check.drop_duplicates(subset=['Source Storage Bin'], keep='last')
must_check.append(data_check)
must_check = must_check.append(data_check)

In [11]:
# Append empty movement data to dataframe for changed SLED because of posting
data_check = data[data.Queue == 'Empty']
data_check = data_check.loc[data_check['Final_Category'] == 3]
data_check = data_check.drop_duplicates(subset=['Source Storage Bin'], keep='last')
must_check.append(data_check)
must_check = must_check.append(data_check)

In [12]:
# Reset numbering index on dataframe
must_check = must_check.reset_index()
must_check

,index,Queue,Source Storage Bin,Dest. Stor. Bin,Act. Qty Dest. BUoM,Src Trgt Qty BUoM,Product,Batch,Shelf Life Exp. Date,Dest. Storage Type,Confirmation Date,Dest_Storage_shift,Product_shift,SLED_shift,Dest_Type_shift,cond1,cond2,cond3,Final_Category
0,25136,INTERNAL,TAHAN-REPACK,TAHAN-REPACK,29,29,971024,2112212002,2022-06-19 00:00:00,8042,2022-01-10 00:00:00,TAHAN-REPACK,971024,2022-06-18 00:00:00,8042,1.0,1.0,1.0,3.0
1,39295,Empty,RF-54-10,Empty,4,0,971022,2112122593,2022-06-10 00:00:00,Empty,2022-01-08 00:00:00,RF-54-10,971022,2022-06-28 00:00:00,PICK,1.0,1.0,1.0,3.0
2,44217,Empty,RF-28-10,Empty,96,0,970761,2112232595,2022-06-21 00:00:00,Empty,2022-01-08 00:00:00,RF-28-10,970761,2022-06-22 00:00:00,PICK,1.0,1.0,1.0,3.0
3,44227,Empty,RH-19-10,Empty,100,0,971025,2112222593,2022-06-20 00:00:00,Empty,2022-01-08 00:00:00,RH-19-10,971025,2022-06-21 00:00:00,PICK,1.0,1.0,1.0,3.0
4,44241,Empty,RF-56-10,Empty,22,0,971023,9000033663,2022-05-14 00:00:00,Empty,2022-01-08 00:00:00,RF-56-10,971023,2022-06-09 00:00:00,PICK,1.0,1.0,1.0,3.0
5,45129,Empty,TA-08-00,Empty,18,0,974067,9000014160,9999-12-31 00:00:00,Empty,2022-01-08 00:00:00,TA-08-00,974067,2022-11-25 00:00:00,PICK,1.0,1.0,1.0,3.0
6,49756,Empty,RF-02-10,Empty,130,0,970594,2109072592,2022-06-04 00:00:00,Empty,2022-01-08 00:00:00,RF-02-10,970594,2022-09-25 00:00:00,PICK,1.0,1.0,1.0,3.0
7,50284,Empty,RB-16-10,Empty,39,0,970293,2112232007,2023-04-22 00:00:00,Empty,2022-01-08 00:00:00,RB-16-10,970293,2023-05-06 00:00:00,PICD,1.0,1.0,1.0,3.0
8,56470,Empty,RC-13-10,Empty,125,0,970928,2112232004,2022-06-21 00:00:00,Empty,2022-01-09 00:00:00,RC-13-10,970928,2022-06-22 00:00:00,PICD,1.0,1.0,1.0,3.0
9,57405,Empty,RH-13-10,Empty,4,0,970546,2111142593,2022-05-13 00:00:00,Empty,2022-01-10 00:00:00,RH-13-10,970546,2022-06-27 00:00:00,PICK,1.0,1.0,1.0,3.0


In [13]:
# Filtering value to racking area
values = ['PICD','PICK','RACK','Empty']
must_check1 = must_check[must_check.Dest_Type_shift.isin(values) == True]
must_check1

,index,Queue,Source Storage Bin,Dest. Stor. Bin,Act. Qty Dest. BUoM,Src Trgt Qty BUoM,Product,Batch,Shelf Life Exp. Date,Dest. Storage Type,Confirmation Date,Dest_Storage_shift,Product_shift,SLED_shift,Dest_Type_shift,cond1,cond2,cond3,Final_Category
1,39295,Empty,RF-54-10,Empty,4,0,971022,2112122593,2022-06-10 00:00:00,Empty,2022-01-08 00:00:00,RF-54-10,971022,2022-06-28 00:00:00,PICK,1.0,1.0,1.0,3.0
2,44217,Empty,RF-28-10,Empty,96,0,970761,2112232595,2022-06-21 00:00:00,Empty,2022-01-08 00:00:00,RF-28-10,970761,2022-06-22 00:00:00,PICK,1.0,1.0,1.0,3.0
3,44227,Empty,RH-19-10,Empty,100,0,971025,2112222593,2022-06-20 00:00:00,Empty,2022-01-08 00:00:00,RH-19-10,971025,2022-06-21 00:00:00,PICK,1.0,1.0,1.0,3.0
4,44241,Empty,RF-56-10,Empty,22,0,971023,9000033663,2022-05-14 00:00:00,Empty,2022-01-08 00:00:00,RF-56-10,971023,2022-06-09 00:00:00,PICK,1.0,1.0,1.0,3.0
5,45129,Empty,TA-08-00,Empty,18,0,974067,9000014160,9999-12-31 00:00:00,Empty,2022-01-08 00:00:00,TA-08-00,974067,2022-11-25 00:00:00,PICK,1.0,1.0,1.0,3.0
6,49756,Empty,RF-02-10,Empty,130,0,970594,2109072592,2022-06-04 00:00:00,Empty,2022-01-08 00:00:00,RF-02-10,970594,2022-09-25 00:00:00,PICK,1.0,1.0,1.0,3.0
7,50284,Empty,RB-16-10,Empty,39,0,970293,2112232007,2023-04-22 00:00:00,Empty,2022-01-08 00:00:00,RB-16-10,970293,2023-05-06 00:00:00,PICD,1.0,1.0,1.0,3.0
8,56470,Empty,RC-13-10,Empty,125,0,970928,2112232004,2022-06-21 00:00:00,Empty,2022-01-09 00:00:00,RC-13-10,970928,2022-06-22 00:00:00,PICD,1.0,1.0,1.0,3.0
9,57405,Empty,RH-13-10,Empty,4,0,970546,2111142593,2022-05-13 00:00:00,Empty,2022-01-10 00:00:00,RH-13-10,970546,2022-06-27 00:00:00,PICK,1.0,1.0,1.0,3.0
11,57428,Empty,RG-03-10,Empty,74,0,970649,2112182593,2022-06-16 00:00:00,Empty,2022-01-10 00:00:00,RG-03-10,970649,2022-06-17 00:00:00,PICK,1.0,1.0,1.0,3.0


In [14]:
print("============================================================================================================================================================================")
print("=================================================================== Product Durability Prevention ==========================================================================")
print("====================================================================== built by Andhyka C.A. ===============================================================================")
print("============================================================================================================================================================================")
print(" ")
print("Good morning! Please do a re-check for storage bin below :")
print(" ")
print(" ")
print("This data based on warehouse task from", must_check1['Confirmation Date'].min().date(), "until now." )
print(np.sort(must_check1['Dest_Storage_shift'].values))
print(" ")
print(" ")
print("Notes : The storage bin above had different Expirement date than before, so there is high possibility the storage bin(s) product have mixed expirement date on each bin.")
print(" ")
print("                                                                                                                                                 Regards, Inventory Control.")
print(" ")
print("============================================================================================================================================================================")

=================================================================== Product Durability Prevention ==========================================================================
====================================================================== built by Andhyka C.A. ===============================================================================
 
Good morning! Please do a re-check for storage bin below :
 
 
This data based on warehouse task from 2022-01-08 until now.
['RB-16-10' 'RC-13-10' 'RF-02-10' 'RF-28-10' 'RF-30-10' 'RF-38-10'
 'RF-48-10' 'RF-54-10' 'RF-56-10' 'RG-03-10' 'RG-05-10' 'RG-28-10'
 'RG-60-10' 'RH-03-10' 'RH-13-10' 'RH-19-10' 'TA-08-00']
 
 
Notes : The storage bin above had different Expirement date than before, so there is high possibility the storage bin(s) product have mixed expirement date on each bin.
 
                                                                                                                                                 Regards, Inventor